# Consolidating Product and Table Metadata into a Single Table

This table can be scheduled to update the metadata table periodically. 

In [0]:
%sql
CREATE TABLE IF NOT EXISTS fnma_product_catalog_jcg.default.product_catalog AS
WITH schema_desciptions AS (
  SELECT
    s.SCHEMA_NAME as Product_Name,
    s.comment as Description,
    t.TAG_NAME,
    t.TAG_VALUE
  FROM
    fnma_product_catalog_jcg.INFORMATION_SCHEMA.SCHEMATA s
      LEFT JOIN fnma_product_catalog_jcg.INFORMATION_SCHEMA.SCHEMA_TAGS t
        ON s.SCHEMA_NAME = t.SCHEMA_NAME
  WHERE
    s.schema_name NOT LIKE '%default%'
    AND s.schema_name NOT LIKE '%information_schema%'
),

schema_table_list AS (
  SELECT product, CAST(to_json(tables_and_descriptions) AS STRING ) AS table_names 
  FROM main_jcg.FNMA.products_tables_descriptions
)
SELECT
  ROW_NUMBER() OVER (ORDER BY A.Product_Name) AS unique_id,
  A.*,
  ifnull(
    B.table_names,
    '[{"table_name":"","description":""}]'
  ) AS table_names
FROM
  schema_desciptions A
    LEFT JOIN schema_table_list B
      ON A.product_name = B.product


-- #spark.sql(sql).write.mode("overwrite").saveAsTable("bircatalog.fnma.updated_fnma_product_catalog_revised")

num_affected_rows,num_inserted_rows


In [0]:
%sql
Select * from fnma_product_catalog_jcg.default.product_catalog;

unique_id,Product_Name,Description,TAG_NAME,TAG_VALUE,table_names
1,acquisition_and_origination_analytics,"## Product Overview This data product organizes data related to the acquisition and origination of loans. It includes tables for **borrower demographics**, **loan applications**, and **loan acquisition records**. Use this product for: - Analyzing loan origination processes - Understanding borrower profiles - Tracking acquisition metrics ## Purpose - **Track trends** in loan acquisition - Analyze **borrower profiles** - Assess **application processes** ## Tables 1. **Loan Acquisition** - Tracks details of loan acquisitions. - Fields: Acquisition Date, Loan Amount, Seller, etc. 2. **Loan Application** - Contains applicant information and application status. - Fields: Application ID, Borrower ID, Application Status, Loan Type, etc. 3. **Borrower Demographics** - Stores demographic details of borrowers. - Fields: Borrower ID, Age, Income, Employment Status, etc. 4. **Credit History** - Records the credit history of borrowers. - Fields: Borrower ID, Credit Score, Historical Defaults, etc. ## Ideal Customers - **Fannie Mae acquisition teams** for portfolio analysis. - **Lenders** to evaluate risk and approve loans. - **Risk modelers** to improve credit risk models. - **Business analysts** to drive data-driven decisions. ## Uses - **Identify emerging origination patterns** - **Gauge credit risk** at the entry point - Understand **borrower diversity and affordability** By utilizing this schema, stakeholders can effectively analyze and interpret critical aspects of the loan origination and acquisition processes.",RemoveAfter,2027-01-31,"[{""table_name"":""loan_acquisition"",""description"":""The table contains data related to loan origination, including details about the borrower and the loan itself. It can be used to analyze lending patterns, assess borrower demographics, and evaluate loan performance. Key information includes loan amounts, borrower income, loan-to-value ratios, and whether the borrower is a first-time homebuyer.""},{""table_name"":""application_table"",""description"":""The table contains data related to loan applications. It includes details such as the application ID, submission date, borrower information, property details, employment history, total debt, monthly income, and the purpose of the loan. This data can be used for analyzing loan application trends, assessing borrower profiles, and evaluating the financial health of applicants.""},{""table_name"":""credit_history"",""description"":""The table contains data related to borrowers' credit profiles. It includes information such as the number of credit accounts, the age of those accounts, payment history, credit inquiries, and the debt-to-income ratio. This data can be used for analyzing borrower creditworthiness, assessing risk for lending decisions, and understanding trends in credit behavior.""},{""table_name"":""borrower_demographics"",""description"":""The table contains demographic and financial information about borrowers. It includes details such as race, ethnicity, gender, date of birth, income, and the number of borrowers associated with each record. This data can be used for analyzing borrower profiles, understanding lending patterns, and assessing the impact of demographic factors on borrowing behavior.""}]"
2,collateral_and_property_intelligence,"## Collateral and Property Intelligence Schema The schema organizes data related to collateral and property intelligence. It encompasses a variety of tables and functions designed to streamline property evaluations, risk assessments, and collateral management. ### Purpose Centralize and analyze property details and valuations for risk and compliance. ### Components **Tables:** - **Property Characteristics:** Details about the property including size, location, and type. - **Property Appraisal:** Data on property assessment and appraised values. - **Unit Rental:** Information on rental units, including

In [0]:
%sql
-- So the vector search index can stay up to date
ALTER TABLE fnma_product_catalog_jcg.default.product_catalog SET TBLPROPERTIES (delta.enableChangeDataFeed = true);